In [4]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 22: Wizard Simulator 20XX ---</h2><p>Little Henry Case decides that defeating bosses with <a href="21">swords and stuff</a> is boring.  Now he's playing the game with a <span title="Being a !@#$% Sorcerer.">wizard</span>.  Of course, he gets stuck on another boss and needs your help again.</p>
<p>In this version, combat still proceeds with the player and the boss taking alternating turns.  The player still goes first.  Now, however, you don't get any equipment; instead, you must choose one of your spells to cast.  The first character at or below <code>0</code> hit points loses.</p>
<p>Since you're a wizard, you don't get to wear armor, and you can't attack normally.  However, since you do <em>magic damage</em>, your opponent's armor is ignored, and so the boss effectively has zero armor as well.  As before, if armor (from a spell, in this case) would reduce damage below <code>1</code>, it becomes <code>1</code> instead - that is, the boss' attacks always deal at least <code>1</code> damage.</p>
<p>On each of your turns, you must select one of your spells to cast.  If you cannot afford to cast any spell, you lose.  Spells cost <em>mana</em>; you start with <em>500</em> mana, but have no maximum limit.  You must have enough mana to cast a spell, and its cost is immediately deducted when you cast it.  Your spells are Magic Missile, Drain, Shield, Poison, and Recharge.</p>
<ul>
<li><em>Magic Missile</em> costs <code>53</code> mana.  It instantly does <code>4</code> damage.</li>
<li><em>Drain</em> costs <code>73</code> mana.  It instantly does <code>2</code> damage and heals you for <code>2</code> hit points.</li>
<li><em>Shield</em> costs <code>113</code> mana.  It starts an <em>effect</em> that lasts for <code>6</code> turns.  While it is active, your armor is increased by <code>7</code>.</li>
<li><em>Poison</em> costs <code>173</code> mana.  It starts an <em>effect</em> that lasts for <code>6</code> turns.  At the start of each turn while it is active, it deals the boss <code>3</code> damage.</li>
<li><em>Recharge</em> costs <code>229</code> mana.  It starts an <em>effect</em> that lasts for <code>5</code> turns.  At the start of each turn while it is active, it gives you <code>101</code> new mana.</li>
</ul>
<p><em>Effects</em> all work the same way.  Effects apply at the start of both the player's turns and the boss' turns.  Effects are created with a timer (the number of turns they last); at the start of each turn, after they apply any effect they have, their timer is decreased by one.  If this decreases the timer to zero, the effect ends.  You cannot cast a spell that would start an effect which is already active.  However, effects can be started on the same turn they end.</p>
<p>For example, suppose the player has <code>10</code> hit points and <code>250</code> mana, and that the boss has <code>13</code> hit points and <code>8</code> damage:</p>
<pre><code>-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --

- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
  Poison deals 3 damage; its timer is now 5.
  Boss attacks for 8 damage.

-- Player turn --

- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
  Poison deals 3 damage; its timer is now 4.
  Player casts Magic Missile, dealing 4 damage.

-- Boss turn --

- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
</code></pre>
<p>Now, suppose the same initial conditions, except that the boss has <code>14</code> hit points instead:</p>
<pre><code>-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 14 hit points
  Player casts Recharge.

-- Boss turn --

- Player has 10 hit points, 0 armor, 21 mana
- Boss has 14 hit points
  Recharge provides 101 mana; its timer is now 4.
  Boss attacks for 8 damage!

-- Player turn --

- Player has 2 hit points, 0 armor, 122 mana
- Boss has 14 hit points
  Recharge provides 101 mana; its timer is now 3.
  Player casts Shield, increasing armor by 7.

-- Boss turn --

- Player has 2 hit points, 7 armor, 110 mana
- Boss has 14 hit points
  Shield's timer is now 5.
  Recharge provides 101 mana; its timer is now 2.
  Boss attacks for 8 - 7 = 1 damage!

-- Player turn --

- Player has 1 hit point, 7 armor, 211 mana
- Boss has 14 hit points
  Shield's timer is now 4.
  Recharge provides 101 mana; its timer is now 1.
  Player casts Drain, dealing 2 damage, and healing 2 hit points.

-- Boss turn --

- Player has 3 hit points, 7 armor, 239 mana
- Boss has 12 hit points
  Shield's timer is now 3.
  Recharge provides 101 mana; its timer is now 0.
  Recharge wears off.
  Boss attacks for 8 - 7 = 1 damage!

-- Player turn --

- Player has 2 hit points, 7 armor, 340 mana
- Boss has 12 hit points
  Shield's timer is now 2.
  Player casts Poison.

-- Boss turn --

- Player has 2 hit points, 7 armor, 167 mana
- Boss has 12 hit points
  Shield's timer is now 1.
  Poison deals 3 damage; its timer is now 5.
  Boss attacks for 8 - 7 = 1 damage!

-- Player turn --

- Player has 1 hit point, 7 armor, 167 mana
- Boss has 9 hit points
  Shield's timer is now 0.
  Shield wears off, decreasing armor by 7.
  Poison deals 3 damage; its timer is now 4.
  Player casts Magic Missile, dealing 4 damage.

-- Boss turn --

- Player has 1 hit point, 0 armor, 114 mana
- Boss has 2 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
</code></pre>
<p>You start with <em>50 hit points</em> and <em>500 mana points</em>. The boss's actual stats are in your puzzle input. What is the <em>least amount of mana</em> you can spend and still win the fight?  (Do not include mana recharge effects as "spending" negative mana.)</p>
</article>


In [5]:
# Specs:
#  1 - player and boss taking alternating turns
#  2 - player goes first
#  3 - the first charactar with hit point <= 0 loses
#  4 - player has no armor and boss's armor ignored
#  5 - for player if armor from spell reduces damage below 1 it is at least 1
#  6 - player must select each of his turns
#  7 - spells cost mana
#  8 - if player can not afford spell he has lost
#  9 - 3ffects apply at the start of both the player's turns and the boss' turns
# 10 - Effects are created with a timer (the number of turns they last);
#      at the start of each turn, after they apply any effect they have,
#      their timer is decreased by one
# 11 - If this decreases the timer to zero, the effect ends
# 12 - You cannot cast a spell that would start an effect which is already active.
#      However, effects can be started on the same turn they end.
# 13 - You start with 50 hit points and 500 mana points
# 14 = Boss starts with Hit Points: 71 and Damage: 10 (In this case)
from functools import cache


@dataclass(frozen=True, order=True)
class Spell:
    name: str
    cost: int
    duration: int
    deals_damage: int = 0
    adds_hitpoints: int = 0
    adds_armor: int = 0
    adds_mana: int = 0

    def decrease_timer_one(self):
        return Spell(
            self.name,
            self.cost,
            self.duration - 1,
            self.deals_damage,
            self.adds_hitpoints,
            self.adds_armor,
            self.adds_mana,
        )


spells = [
    # Magic Missile:
    #   - costs 53 mana.
    #   -  It instantly does 4 damage.
    Spell("Magic Missile", 53, 0, deals_damage=4),
    # Drain:
    #   - costs 73 mana.
    #   - It instantly does 2 damage and heals you for 2 hit points.
    Spell("Drain", 73, 0, deals_damage=2, adds_hitpoints=2),
    # Shield:
    #   - costs 113 mana.
    #   - It starts an effect that lasts for 6 turns.
    #     While it is active, your armor is increased by 7.
    Spell("Shield", 113, 6, adds_armor=7),
    # Poison
    #   - costs 173 mana.
    #   - It starts an effect that lasts for 6 turns.
    #   - At the start of each turn while it is active, it deals the boss 3 damage.
    Spell("Poison", 173, 6, deals_damage=3),
    # Recharge
    #   - costs 229 mana.
    #   - It starts an effect that lasts for 5 turns.
    #     At the start of each turn while it is active, it gives you 101 new mana.
    Spell("Recharge", 229, 5, adds_mana=101),
]


class Game_I:
    def __init__(self, me, boss, spells) -> None:
        self.me_hitpoints = me["hitpoints"]
        self.me_mana = me["mana"]
        self.boss_hitpoints = boss["hitpoints"]
        self.boss_damage = boss["damage"]
        self.spells = spells
        self.min_cost = min(s.cost for s in spells)

    @cache
    def my_turn(
        self,
        me_hitpoints: int,
        me_mana: int,
        boss_hitpoints: int,
        active_spells: tuple[Spell, ...],
        mana_spent: int,
    ) -> None:

        _, me_hitpoints, me_mana, boss_hitpoints, active_spells = self.apply_effects(
            me_hitpoints, me_mana, boss_hitpoints, active_spells
        )

        if boss_hitpoints <= 0:
            self.least_mana_spent = min(self.least_mana_spent, mana_spent)
            return

        if (
            me_hitpoints <= 0
            or me_mana < self.min_cost
            or me_mana >= self.least_mana_spent
        ):
            return

        for spell in spells:
            if (
                me_mana >= spell.cost
                and mana_spent + spell.cost < self.least_mana_spent
                and all(spell.name != s.name for s in active_spells)
            ):
                self.turn_boss(
                    me_hitpoints,
                    me_mana - spell.cost,
                    boss_hitpoints,
                    active_spells + (spell,),
                    mana_spent + spell.cost,
                )

    @cache
    def turn_boss(
        self,
        me_hitpoints: int,
        me_mana: int,
        boss_hitpoints: int,
        active_spells: tuple[Spell],
        mana_spent: int,
    ) -> None:

        my_armor, me_hitpoints, me_mana, boss_hitpoints, active_spells = (
            self.apply_effects(me_hitpoints, me_mana, boss_hitpoints, active_spells)
        )

        if boss_hitpoints <= 0:
            self.least_mana_spent = min(self.least_mana_spent, mana_spent)
            return

        me_hitpoints -= max(self.boss_damage - my_armor, 1)
        if me_hitpoints <= 0:
            return

        return self.my_turn(
            me_hitpoints,
            me_mana,
            boss_hitpoints,
            active_spells,
            mana_spent,
        )

    def apply_effects(
        self,
        me_hitpoints: int,
        me_mana: int,
        boss_hitpoints: int,
        active_spells: tuple[Spell, ...],
    ) -> tuple[int, int, int, tuple[Spell, ...]]:
        my_armor = 0
        still_active_spells = []

        for spell in active_spells:
            me_hitpoints += spell.adds_hitpoints
            me_mana += spell.adds_mana
            my_armor += spell.adds_armor
            boss_hitpoints -= spell.deals_damage

            if spell.duration > 1:
                still_active_spells.append(spell.decrease_timer_one())

        return (
            my_armor,
            me_hitpoints,
            me_mana,
            boss_hitpoints,
            tuple(still_active_spells),
        )

    def least_mana_to_win(self):
        self.least_mana_spent = inf  # Even though returning the value sis better style,
        # This gives you the possibility to prune better and
        # and so this caching

        self.my_turn(
            self.me_hitpoints, self.me_mana, self.boss_hitpoints, frozenset(), 0
        )
        return self.least_mana_spent


me = {"hitpoints": 50, "mana": 500}
boss = {"hitpoints": 71, "damage": 10}

Game_I(me, boss, spells).least_mana_to_win()

1824

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1824</code>.</p><article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>On the next run through the game, you increase the difficulty to <em>hard</em>.</p>
<p>At the start of each <em>player turn</em> (before any other effects apply), you lose <code>1</code> hit point. If this brings you to or below <code>0</code> hit points, you lose.</p>
<p>With the same starting stats for you and the boss, what is the <em>least amount of mana</em> you can spend and still win the fight?</p>
</article>

</main>


In [6]:
from typing import override


class Game_II(Game_I):

    @override
    def my_turn(
        self,
        me_hitpoints: int,
        me_mana: int,
        boss_hitpoints: int,
        active_spells: tuple[Spell, ...],
        mana_spent: int,
    ) -> None:
        if me_hitpoints - 1 > 0:
            super().my_turn(
                me_hitpoints - 1,
                me_mana,
                boss_hitpoints,
                active_spells,
                mana_spent,
            )


Game_II(me, boss, spells).least_mana_to_win()

1937

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1937</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="22/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Wizard+Simulator+20XX%22+%2D+Day+22+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F22&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Wizard+Simulator+20XX%22+%2D+Day+22+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F22';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
